Create Dataset

In [1]:
import pandas as pd
import numpy as np

first_df=pd.read_csv('Arrests_cleaned.csv')

In [2]:
import requests, json
r = requests.get('https://data.cityofchicago.org/resource/crimes.json?$limit=600000')
data = r.json()
sec_df = pd.DataFrame(data)
sec_df = sec_df[['case_number','ward','district']]

In [3]:
merged_df = pd.merge(first_df, sec_df, on="case_number")
print(len(merged_df))
merged_df.head(1)

68416


,id,case_number,date_month,date_day,date_year,time,meridiem,race,charge_1_statute,charge_1_description,...,charge_4_statute,charge_4_description,charge_4_type,charge_4_class,CHARGES STATUTE,CHARGES DESCRIPTION,CHARGES TYPE,CHARGES CLASS,ward,district
0,30039812,JE183770,3,30,2021,12:10:00,AM,WHITE HISPANIC,720 ILCS 5.0/12-3.2-A-1,DOMESTIC BATTERY - BODILY HARM,...,NaN,NaN,NaN,NaN,720 ILCS 5.0/12-3.2-A-1 | | |,DOMESTIC BATTERY - BODILY HARM | | |,M | | |,A | | |,14,008


In [4]:
races = {'BLACK': 1, 'WHITE': 2, 'WHITE HISPANIC': 3, 'BLACK HISPANIC': 4, 'ASIAN / PACIFIC ISLANDER': 5, 'AMER INDIAN / ALASKAN NATIVE': 6, 'UNKNOWN / REFUSED': 7}
merged_df['race'] = merged_df['race'].apply(lambda x: races[x])
merged_df = merged_df[['race','ward','district','date_day','date_year','charge_1_type']].dropna()
merged_df.head(1)


,race,ward,district,date_day,date_year,charge_1_type
0,3,14,008,30,2021,M


In [5]:
from sklearn.model_selection import train_test_split

x = merged_df[['race','ward','district','date_day','date_year']]
y = merged_df[['charge_1_type']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

KNN

In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

knn = KNeighborsClassifier(n_neighbors=5)
scalers = [MinMaxScaler(), StandardScaler(), PCA()]
for scaling in scalers:
    pipe = make_pipeline(scaling, knn)
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    length = len(y_pred)
    correct = 0
    concat = np.array(y_test['charge_1_type'])
    for y in range(length):
        if concat[y] == y_pred[y]:
            correct += 1
    score = correct/length
    print("score = ", score)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


score =  0.5600789358280953


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


score =  0.5604443794766847


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


score =  0.5595673147200702


Neural Network

In [6]:
from statistics import mean 

from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [7]:

net_classifier = MLPClassifier()
net_classifier.fit(x_train,y_train['charge_1_type'].tolist())
classifier_preds = net_classifier.predict(x_test)
score = net_classifier.score(x_test,y_test)
print(score)


0.44949568776494664


In [11]:
scalers = [MinMaxScaler(), StandardScaler(), PCA()]
scores = []
for scaling in scalers:
    pipe = make_pipeline(scaling, net_classifier)
    pipe.fit(x_train,y_train['charge_1_type'].tolist())
    scores.append(pipe.score(x_test,y_test))

print("MinMaxScore =",scores[0])
print("StandardScaler =",scores[1])
print("PCA =",scores[2])

MinMaxScore = 0.5770355211226429
StandardScaler = 0.5843443940944306
PCA = 0.5811284899868441


In [18]:
parameters = {
    # 'solver': ['lbfgs', 'sgd', 'adam'],
    # 'activation' : ['identity', 'logistic', 'tanh', 'relu'],
    # 'alpha' : [0.001, 0.01,0.0001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009],
    'learning_rate' : ['constant', 'invscaling', 'adaptive'],
    # 'learning_rate_init' : [0.001,0.002,0.0001,0.0011,0.0009],
    'random_state':[0,100,200,300,400,500],
}

search = GridSearchCV(net_classifier,parameters)
search.fit(x_train,y_train['charge_1_type'].tolist())

best = search.best_estimator_
best_score = best.score(x_test,y_test)
print(best_score)
# scores = cross_val_score(best,x_test,y_test,cv=5)
# avg_score = mean(scores)
# print(avg_score)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.5097208010524777


In [19]:
scores = []
for scaling in scalers:
    pipe = make_pipeline(scaling, best)
    pipe.fit(x_train,y_train['charge_1_type'].tolist())
    scores.append(pipe.score(x_test,y_test))

print("MinMaxScore =",scores[0])
print("StandardScaler =",scores[1])
print("PCA =",scores[2])

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MinMaxScore = 0.5791550942844613
StandardScaler = 0.5768162549334892
PCA = 0.5845636602835843
